# Iterative Lattice Analysis Example

Install with `pip install iterative-lattice-analysis`

`setup_and_execute_iteration_method` from the `iteration_functions` module is the main method for using the iterative method on any python function that represents the one turn map.

A one turn map can be create from a madx sequence with `create-ptc-files` from the `madx-ptc-utils` package. From the example directory run the following command.

`$ create-ptc-files example.madx --ptc-order 10 --ptc-dim 4`

A folder called "out" will be created with another folder "example" inside. This is to prevent overwriting of other lattices.

In [ ]:
from madx_ptc_utils.python_from_ptc import python_from_ptc
import numpy as np
import sympy
import pathlib
import matplotlib.pyplot as plt
from iterative_lattice_analysis.iteration_functions import (
    setup_and_execute_iteration_method,
)

%matplotlib widget

## Lattice Parameters

In [ ]:
dim = 2  # Spacial dimension, i.e. 4-D phase space
order = 3  # Order of the square matrix treatment on the coordinates
res = None  # Any nearby resonances to avoid ex: [1, 2] is for Qx + 2Qy = l where l is an integer
x_range = (-0.035, 0.030)  # Range of the x coordinates
y_range = (0, 0.013)  # Range of the y coordinates
npos = (150, 100)  # Number of points in the x and y coordinates
nangle = int(2**4)  # Number of angles to consider when sampling each initial condition
niters = 20  # Number of iterations to perform for the iterative method
cores = 32  # Number of cores to use for the parallel computation

# Folder to save the results of the iteration method
results_folder = pathlib.Path("./iteration-results/")

You can import the one turn map directly as a python object by passing the folder into the `python_from_ptc` function from the `madx-ptc-utils` package.

In [ ]:
# Folder with the PTC files i.e. the output of `create-ptc-files` function
function_folder = pathlib.Path("./out/example")

func = python_from_ptc(function_folder)

x, px, y, py = sympy.symbols("x p_x y p_y")

nx, npx, ny, npy = func(x, px, y, py)

display(nx)

But you could also pass the folder into the iteration method instead of a python function.

In [ ]:
setup_and_execute_iteration_method(
    dim=dim,
    order=order,
    res=res,
    x_range=x_range,
    y_range=y_range,
    number_position_samples=npos,
    number_angle_samples=nangle,
    number_iterations=niters,
    function=function_folder,  # Takes the folder with the PTC files or a function
    results_folder=results_folder,
    newton_inverse=True,  # Use numerical inverse for the square matrix
    cores=cores,
)

Load the data found in the results folder

In [ ]:
U = np.loadtxt(results_folder / "U.txt", dtype=np.complex128)
linear_tunes = np.loadtxt(results_folder / "linear_tunes.txt", dtype=np.float64)
xiniarr = np.loadtxt(results_folder / "xini.txt", dtype=np.float64)
yiniarr = np.loadtxt(results_folder / "yini.txt", dtype=np.float64)
freq_results = np.loadtxt(results_folder / "freq.txt", dtype=np.float64)
error_results = np.loadtxt(results_folder / "error.txt", dtype=np.float64)
tune_results = freq_results / (2 * np.pi)

Plot the data

In [ ]:
fig, ax = plt.subplots()

ax.set_title("Convergence Map")
ax.set_xlabel(r"$x$")
ax.set_ylabel(r"$y$")

error_exponent = np.log10(error_results + 1e-16)
colorbar_label = r"$\min (\log_{10} {(||{W_{n+1} - W_n}||+10^{-16})})$"

im = ax.scatter(
    xiniarr,
    yiniarr,
    c=error_exponent,
    s=7,
    marker="s",
    cmap="jet",
    vmax=np.nanmax(error_exponent),
    vmin=np.nanmin(error_exponent),
)


fig.colorbar(im, ax=ax, label=colorbar_label)
fig.savefig(results_folder / "convergence-map.png", dpi=300)

print("Max error:", np.nanmax(error_results))
print("Min error:", np.nanmin(error_results))

In [ ]:
fig, ax = plt.subplots()

ax.set_title("Convergence Map Tune Space")
ax.set_xlabel("$Q_x$")
ax.set_ylabel("$Q_y$")

im = ax.scatter(
    tune_results[:, 0],
    tune_results[:, 1],
    c=error_exponent,
    cmap="jet",
    vmax=np.nanmax(error_exponent),
    vmin=np.nanmin(error_exponent),
    s=10,
)

ax.plot(
    *linear_tunes,
    marker="x",
    label="linear tune",
    color="black",
    linestyle="None",
    markersize=10,
)

fig.legend()

fig.colorbar(im, ax=ax, label=colorbar_label)

fig.savefig(results_folder / "convergence-map-tune-space.png", dpi=300)